# Equations of Motion for a 6 DOF Aircraft

Based on:
- [Aircraft 6-DOF Equations and Coding in Python](https://www.youtube.com/watchv=hr_PqdkG6XY&list=PLcmbTy9X3gXs4JVXYucrMz5bJ4ZuaEGJ_&index=1)
- AIRCRAFT CONTROL AND SIMULATION 3rd Ed. Dynamics, Controls Design, & Autonomous Systems - Stevens, Lewis, Johnson (Wiley)

In [2]:
import numpy as np
from sympy import *

<img src="reference_frames.png" alt="Ref" width="600" />



<img src="flat_earth_frame.png" alt="flat earth" width="600" />

This simulation will use flat-Earth equations, describing motion over a small area of a nonrotating Earth, with constant gravity.

Earth centered, earth fixed CS: [x_e, y_e, z_e]

North East Down (NED) CS: [x_n, y_n, z_n]
- NED rotates wrt aircraft according to curvature of earth
- Tangent to earth's surface


### Reference Frames and Coordinate Systems

<img src="coordinate systems.png" alt="flat earth" width="600" />

Assumptions:
- The Earth frame is an inertial reference frame.
- Position is measured in a tangent-plane coordinate system, tp.
- The gravity vector is normal to the tangent plane and constant in magnitude.


\begin{align*}
r^n_{CM/T} &: \text{position vector, aircract CM wrt point T, resolved in NED CS} \\
v^b_{CM/e} &: \text{velocity vector, aircraft CM wrt inertial frame e, resolved in body fixed CS b} \\
\omega_{b/e}^b &: \text{angular velocity vector, aircraft body fram wrt inertial frame, resolved in body CS} \\
& \text {**All state derivates will taken in body frame**}
\end{align*}

## Translational Equations

In [15]:
u,v,w,p,q,r,G,psi,theta,phi,F_x,F_y,F_z,m = symbols('u,v,w,p,q,r,G,psi,theta,phi,F_x,F_y,F_z,m')

vel_b = Matrix([u,v,w])
omega_b = Matrix([p,q,r])
F_a = Matrix([F_x,F_y,F_z])
g_n = Matrix([0,0, G])


Rx = Matrix([[1, 0, 0],
             [0, cos(phi), sin(phi)],
             [0, -sin(phi), cos(phi)]])

Ry = Matrix([[cos(theta), 0, -sin(theta)],
             [0, 1, 0],
             [sin(theta), 0, cos(theta)]])

Rz = Matrix([[cos(psi), sin(psi), 0],
             [-sin(psi), cos(psi), 0],
             [0, 0, 1]])

C_bn = Rx @ Ry @ Rz

acc_cm = 1/m*F_a + C_bn @ g_n - (omega_b.cross(vel_b))

acc_cm

Matrix([
[         F_x/m - G*sin(theta) - q*w + r*v],
[F_y/m + G*sin(phi)*cos(theta) + p*w - r*u],
[F_z/m + G*cos(phi)*cos(theta) - p*v + q*u]])

***EXPAND DERIVATION***

$$
\dot v^b_{CM/e} = \frac{1}{m} F^b_{A,T} + C_{b/n}g^n - (\omega ^b_{b/e} \times v^b_{CM/e})
$$

$\dot v^b_{CM/e} $: Acceleration

$\frac{1}{m} F^b_{A,T}$: Acceleration due to aerodynamic and thrust forces

- m: mass
- F: Aerodynamic and thrust forces

$C_{b/n}g^n$: Acceleration due to gravity

- C: Direction cosine matrix rotation of NED to body CS (Euler angles)
- g: gravity resolved in NED CS (multiplied by the cosine matrix to change to body frame)

$\omega ^b_{b/e} \times v^b_{CM/e}$: Acceleration due to rotation of velocity vector

- $\omega$: Angular velocity of body frame wrt earth frame
- v: velocity of cm wrt earth frame (state)


\begin{align*}
    v^b_{CM/e} & = \begin{bmatrix} u \\ v \\ w\end{bmatrix} \qquad 
    \omega ^b_{b/e} = \begin{bmatrix} p \\ q \\ r\end{bmatrix} \\
    F^b_{A,T} & = \begin{bmatrix} F_{xb} \\ F_{yb} \\ F_{zb}\end{bmatrix} \qquad g^n = \begin{bmatrix} 0 \\ 0 \\ G\end{bmatrix} \\
\end{align*}

$$
C_{b/n} = \begin{bmatrix} cos(\theta)cos(\psi) & cos(\theta)sin(\psi) & -sin(\theta) \\
                                -cos(\phi)sin(\psi) + sin(\phi)sin(\theta)cos(\psi) & cos(\phi)cos(\psi) + sin(\phi)sin(\theta)sin(\psi) & sin(\phi)cos(\theta) \\ 
                                sin(\phi)sin(\psi) + cos(\phi)sin(\theta)cos(\psi) & -sin(\phi)cos(\psi) + cos(\phi)sin(\theta)sin(\psi) & cos(\phi)cos(\theta) \end{bmatrix}
$$

$$
C_{b/n}g^n = \begin{bmatrix} -Gsin(\theta) \\ Gsin(\phi)cos(\theta) \\ Gcos(\phi)cos(\theta) \end{bmatrix}
$$
$$
\therefore \dot v^b_{CM/e}= 
    \begin{bmatrix}
        \dot u\\ \dot v \\ \dot w
    \end{bmatrix} =
    \begin{bmatrix} 
        \frac{1}{m}F_{xb} -Gsin(\theta) - wq + vr \\
        \frac{1}{m}F_{yb} + Gsin(\phi)cos(\theta) - ur + wp \\
        \frac{1}{m}F_{zb} + Gcos(\phi)cos(\theta) - vp + uq 
    \end{bmatrix}
$$

## Rotational Equations

In [78]:
J_x,J_y,J_z,J_xz,L,M,N = symbols('J_x,J_y,J_z,J_xz,L,M,N')

M_AT = Matrix([L, M, N])
J_b = Matrix([[J_x, 0 , J_xz],
              [0, J_y, 0],
              [J_xz,0,J_z]])

ang_acc_cm = (J_b.inv() @ (M_AT - omega_b.cross(J_b @ omega_b)))

ang_acc_cm

Matrix([
[-J_xz*(-J_y*p*q + N + q*(J_x*p + J_xz*r))/(J_x*J_z - J_xz**2) + J_z*(J_y*q*r + L - q*(J_xz*p + J_z*r))/(J_x*J_z - J_xz**2)],
[                                                                         (M + p*(J_xz*p + J_z*r) - r*(J_x*p + J_xz*r))/J_y],
[ J_x*(-J_y*p*q + N + q*(J_x*p + J_xz*r))/(J_x*J_z - J_xz**2) - J_xz*(J_y*q*r + L - q*(J_xz*p + J_z*r))/(J_x*J_z - J_xz**2)]])


$$
\dot \omega^b_{b/e} = (J^b)^{-1} \left[M^b_{A,T} - (\omega^b_{b/e} \times v^b_{CM/e})J^b \omega^b_{b/e} \right]
$$

$\dot \omega^b_{b/e}$: angular acceleration

$M^b_{A,T}$: angular acceleration due to moments from aerodynamic and thrust forces, Moment vector

$(\omega^b_{b/e} \times v^b_{CM/e})J^b \omega^b_{b/e}$: angular acceleration due to rotation of angular momentum vector

- $\omega^b_{b/e}$: angular velocity of body CS wrt earth frame
- $v^b_{CM/e}$: velocity of CM wrt earth frame
- $J^b$: inertia matrix


\begin{align*}
    M^b_{A,T} & = \begin{bmatrix} L \\ M \\ N \end{bmatrix} \\
    J^b = 
    \begin{bmatrix} J_{xx} & J_{xy} & J_{xz} \\
                    J_{yx} & J_{yy} & J_{yz} \\
                    J_{zx} & J_{zy} & J_{zz} \end{bmatrix} &=
    \begin{bmatrix} J_{x} & 0 & J_{xz} \\
                    0 & J_{y} & 0 \\
                    J_{xz} & 0 & J_{z} \end{bmatrix}
\end{align*}

- assuming plane is symmetric about xz plane, due to the symmetry the yz and xy products of inertia cancel out (i.e. $J_{yz}, J_{xy} = 0$)


$$
\therefore \dot \omega^b_{b/e} = \begin{bmatrix} \dot p \\ \dot q \\ \dot r \end{bmatrix} = \begin{bmatrix}
\frac{J_{xz} (J_x-J_y+J_z)pq - \left[J_z(J_z-J_y) + J^2_{xz} \right]qr + J_z L + J_{xz}N}{J_xJ_z-J^2_{xz}} \\
\frac{(J_z - J_x)rp - J_{xz}(p^2-r^2)+M}{J_y} \\
\frac{-J_{xz}(J_x-J_y+J_z)qr + \left[J_x(J_x-J_y) +J^2_{xz}  \right]pqr + J_{xz}L + J_x N}{J_xJ_z - J^2_{xz}}
\end{bmatrix}
$$

